In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import MDAnalysis as mda
from MDAnalysis.analysis.align import alignto
import yaml
import pandas as pd
import tqdm

# Binding of the Ubiquitin UIM1 System

This notebook demonstrates how to simulate the two bound states of the ubiquitin uim1 system. We will look in detail at the following steps

1. Setup
1. Equilibration
1. Production Run
1. Post Processing
1. Analysis

# Setup

To create a **cplx** file, the input for complexes++, we are going to use the `pycomplexes convert` command.
First we define all topologies in the system. In our case each topology consist of one rigid domain. We will
do this by writing a **binding.top** file that *pycomplexes* understands.

In [ ]:
%%writefile binding.top
# Dimensions of PBC box in angstrom
box: [100, 100, 100]
topology:
    # Definition of the first topology named ubiquitin
    Ubiquitin:
        # Take coordinates from this file
        coordinate-file: 1Q0W.pdb
        # now we define the domains in this topology
        domains:
            ubiquitin:
                type: rigid
                selection: 'protein and name CA and segid B'
    # We also need to define the UIM1 topology
    UIM1:
        coordinate-file: 1Q0W.pdb
        domains:
            uim1:
                type: rigid
                selection: 'protein and name CA and segid A'

The **binding.top** file can now be converted. As a forcefield we are going to use the *KimHummer* parameters from the original publication.

In [ ]:
!pycomplexes convert binding.top equilibrate.cplx --forcefield=KimHummer

Now a new file called `equilibrate.cplx` should exist.

In [ ]:
!ls

# Equilibration

We could start the simulation directly from the crystal structure. However we would like to know exactly how the 
bound state in the crystal structure looks like in the KimHummer forcefield. Therefore we will to an equilibration 
run first at only 100 Kelvin.

Before we start the simulation we have to write a configuration file to tell *complexes++* which Monte-Carlo algorithm to chose.

In [ ]:
%%writefile equilibrate.conf
# CPLX file containing coordinates and forcefield
structure: equilibrate.cplx
# Settings of Monte-Carlo Algorithm
montecarlo:
    # We use the nvt ensemble algorithm
    algorithm: nvt
    # with the metropolis acceptance function and a temperature of 100 Kelvin
    algorithm-params:
        accept-func: metropolis
        temperatur: 100
    # We always need to specify a seed. The same seed will yield identical runs
    seed: 3333
    # Because we only have 2 domains it's better not to use the short-range-cutoff
    short-range-cutoff:
        enable: False
# Last we set some parameters for IO
output:
    log: complexes.log
    file: equilibrate.xtc
    # Save a structure every 50 sweeps
    freq: 50
    # Save a total of 50 structures
    nstructures: 50
    stat-file: equilibrate.stat
    # Do not write restart files
    restart-freq: -1

In [ ]:
pwd

In [ ]:
!complexes++ --config=equilibrate.conf

We can have a look at the time evolution of the total energy. Because we run at very low temperatures we expect the initial energy to quickly drop.

In [ ]:
stat = pd.read_csv('equilibrate.stat', index_col=0)

f, ax = plt.subplots()

ax.plot(stat.index, stat.energy, '.-')
ax.set(xlabel='frame', ylabel='Energy [kT]')

plt.tight_layout()

# Production Run

We can now also start from the bound state identified in the equilibration run.
`pycomplexes` has a command for this which automatically choses the last frame in a given trajectory.

In [ ]:
!pycomplexes equilibration equilibrate.cplx equilibrate.xtc equilibrate_reference.pdb production.cplx

We now have to write a new configuration file. We now run at room temperature and use different output file.

In [ ]:
%%writefile production.conf
structure: production.cplx
montecarlo:
    algorithm: nvt
    algorithm-params:
        accept-func: metropolis
        temperatur: 300
    seed: 4242
    short-range-cutoff:
        enable: False
output:
    log: complexes.log
    file: production.xtc
    freq: 20
    nstructures: 1000
    stat-file: production.stat
    restart-freq: -1

In [ ]:
!complexes++ --config=production.conf

# Post-Processing

The raw trajectory can be confusing to look at due to PBC artifacts, like jumps at the box boundary. As a post processing step we are going to center the ubiquitin domain in the box and align all frames to the initial frame. This is going to remove jumps due to PBC artifacts.

In [ ]:
def apply_pbc(doms, box):
    """Ensure all doms are in the PBC box"""
    for d in doms:
        d.translate(-np.floor(d.centroid() / box) * box)

In [ ]:
reference = 'production_reference.pdb'
trajectory = 'production.xtc'
out = 'production_align.xtc'
        
# Prepare input and output
ref = mda.Universe(reference)
trj = mda.Universe(reference, trajectory)
ubiquitin = 'segid B'

# prepare domains selections
doms = [s.atoms for s in trj.segments]
target = trj.select_atoms(ubiquitin)
box = target.dimensions[:3]

# Center target selection in reference
ref_target = ref.select_atoms(ubiquitin)
ref_target.translate(-ref_target.centroid() + .5*box)

with mda.Writer(out, ref.atoms.n_atoms) as w:
    for ts in tqdm.tqdm(trj.trajectory):
        # Move target to center & correct for PBC changes
        # to keep distance under pbc unchanged.
        trj.atoms.translate(-target.centroid() + .5 * box)
        apply_pbc(doms, box)
        # Alignment of all atoms can happen now. It can only 
        # be done now because the alignment might move other 
        # domains outside of the simulation box again!
        alignto(trj, ref, select=ubiquitin)
        w.write(trj)

# Analysis

Lastly, we analyze the bound state of the system. Two natively bound states exist. 
Firstly, we will look at the energy of the system.

In [ ]:
# The *.stat files are CSV formatted and can be read with pandas
stat = pd.read_csv('production.stat', index_col=0)

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=plt.figaspect(.5))

ax1.plot(stat.index, stat.energy)
ax1.set(xlabel='frame', ylabel='energy [kT]', title='timeseries')

ax2.hist(stat.energy, bins=50, density=True)
ax2.set(xlabel='energy [kT]', title='histogram')

plt.tight_layout()

From the energy alone we can see that there is a bound state with a minimal energy of -20 kT. The most populated state has around -15 kT. But we do not see two distinct states here. The peak at 0 kT is the unbound state, which we are not interested in. Another variable we can look at is the RMSD to our bound structure. For this we will use the MDAnalysis package. The `RMSD` function will automatically choose the first frame as a reference 
for the RMSD calculations.

In [ ]:
u = mda.Universe('production_reference.pdb', 'production_align.xtc')
rmsd = mda.analysis.rms.RMSD(u).run()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=plt.figaspect(.5))

ax1.plot(rmsd.rmsd[:, 0], rmsd.rmsd[:, 2])
ax1.set(xlabel='frame', ylabel=r'rmsd [$\AA$]', title='timeseries')

ax2.hist(rmsd.rmsd[:, 2], bins=50, density=True)
ax2.set(xlabel=r'rmsd [$\AA$]', title='histogram')

plt.tight_layout()

We observe two peaks in the histogram of the RMSD.
Maybe we can get a clearer picture by plotting the energy and RMSD together.

In [ ]:
f, ax = plt.subplots()

ax.scatter(rmsd.rmsd[:, 2], stat.energy, edgecolor='none', alpha=0.5)
ax.set(xlabel=r'RMSD $[\AA]$', ylabel='energy [kT]')

plt.tight_layout()

Now we can clearly see the two bound states. The first has a minimum at -20 kT, 
the second has a minimum at -15 kT. 